In [1]:
import json

with open('./task4.json') as f:
    data = json.load(f)


In [ ]:
# If use Together API
from together import Together

your_model_name = 'xxxxxxxxxxxxxxxxxxxxxxxx'
your_api_key = 'xxxxxxxxxxxxxxxxxxxxxxxx'
client = Together(api_key=your_api_key)

In [ ]:
for draft in data:
    statements = draft['Statements']
    for statement in statements:
        country = statement['country']
        system_prompt = f"""
        Assume you are the representative of {country} in UNSC, given the content of a UNSC draft resolution, generate a statement that you would make in the meeting.
        """
        user_prompt = f"""
        Draft resolution: {draft['Content']}
        Your statement:
        """
        response = client.chat.completions.create(
            model=your_model_name,
            temperature=0.0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        statement['generation'] = response.choices[0].message.content
    draft['Statements'] = statements

In [ ]:
statements = []
generations = []

for draft in data:
    for statement in draft['Statements']:
        statements.append(statement['statement'])
        generations.append(statement['generation'])

In [ ]:
# average ROUGE
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

rouge_scores = []
for i in range(len(statements)):
    scores = scorer.score(statements[i], generations[i])
    rouge_scores.append(scores['rougeL'].fmeasure)

print('Average ROUGE-L:', sum(rouge_scores) / len(rouge_scores))

In [ ]:
# average Jaccard similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score

jaccard_scores = []
vectorizer = CountVectorizer(binary=True)

for i in range(len(statements)):
    X = vectorizer.fit_transform([statements[i], generations[i]]).toarray()
    jaccard_scores.append(jaccard_score(X[0], X[1]))

print('Average Jaccard similarity:', sum(jaccard_scores) / len(jaccard_scores))

In [ ]:
# average Cosing Similarity (TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tf_cosine_scores = []
for i in range(len(statements)):
    tfidf = TfidfVectorizer().fit_transform([statements[i], generations[i]])
    tf_cosine_scores.append(cosine_similarity(tfidf[0], tfidf[1])[0][0])

print('Average Cosine Similarity (TF-IDF):', sum(tf_cosine_scores) / len(tf_cosine_scores))

In [ ]:
from sentence_transformers import SentenceTransformer, util

device = 'cuda:1'
model = SentenceTransformer('stsb-roberta-large', device=device)
cosine_scores = []
for i in range(len(statements)):
    embeddings = model.encode([statements[i], generations[i]])
    cosine_scores.append(util.pytorch_cos_sim(embeddings[0], embeddings[1]).item())

print('Average Cosine Similarity (BERT):', sum(cosine_scores) / len(cosine_scores))
